In [40]:
import pandas  as pd
import numpy as np

In [41]:
directors=pd.read_csv('/Users/macbook/Desktop/BA6/Database/db2018imdb/directors.csv', sep=',')
people_actors=pd.read_csv('/Users/macbook/Desktop/BA6/Database/newfiles/people.csv', sep=',')

In [42]:
cols_to_check = ['ClipIds','Roles','AddInfos']
directors[cols_to_check] = directors[cols_to_check].replace({'\[':'','\]':''},regex=True)

directors = directors.replace('', np.nan, regex=True)

directors.drop_duplicates(keep='first', inplace=True)
directors.drop_duplicates(subset=['FullName'], keep='first', inplace=True)

In [43]:
result = pd.merge(directors, people_actors, how='left', on='FullName')
result = result[np.isnan(result['pid'])]
result['pid']=(result.FullName).rank(method='dense')+people_actors['pid'].max()
result[['pid']] = result[['pid']].astype(int)
result= result[['pid','FullName']]
people = pd.concat([people_actors, result])

In [45]:
people.to_csv('people.csv', sep=',', encoding='utf-8', index=False)

In [47]:
direct = pd.merge(directors, people, how='left', on='FullName')

In [49]:
direct = direct.drop(['FullName'],axis=1)

direct[['ClipIds']] = direct[['ClipIds']].astype(str)
direct[['AddInfos']] = direct[['AddInfos']].astype(str)
direct[['Roles']] = direct[['Roles']].astype(str)

def splitDataFrameList(df,target_column,separator):
    ''' df = dataframe to split,
    target_column = the column containing the values to split
    separator = the symbol used to perform the split
    returns: a dataframe with each entry for the target column separated, with each element moved into a new row. 
    The values in the other columns are duplicated across the newly divided rows.
    '''
    def splitListToRows(row,row_accumulator,target_column,separator):
        split_row = []
        i=0
        for c in target_column:
            split_row.append(row[c].split(separator))
            i+=1
            
        for s in range (0,len(split_row[0])):
            j=0
            new_row = row.to_dict()
            for c in target_column:
                new_row[c] = split_row[j][s]
                j+=1
            row_accumulator.append(new_row)
    new_rows = []
    df.apply(splitListToRows,axis=1,args = (new_rows,target_column,separator))
    global new_df
    new_df = pd.DataFrame(new_rows)
    return new_df

direct = splitDataFrameList( direct, ['ClipIds','AddInfos','Roles'] , '|')

In [50]:
direct = direct.replace('', np.nan, regex=True)
direct = direct.replace('nan', np.nan, regex=True)

direct = direct[['pid', 'ClipIds', 'Roles', 'AddInfos']]

direct.drop_duplicates(subset=['pid','ClipIds'], keep=False)

,pid,ClipIds,Roles,AddInfos
0,2006887,51264,NaN,NaN
1,2006887,742507,NaN,NaN
2,2006887,960777,NaN,NaN
3,61,348914,NaN,NaN
4,61,3600723,NaN,NaN
5,61,3600724,NaN,NaN
6,61,3600725,NaN,NaN
7,67,714863,NaN,NaN
8,85,1799926,NaN,NaN
9,2006888,1228184,NaN,NaN


In [51]:
direct.to_csv('direct.csv', sep=',', encoding='utf-8', index=False)